# Revisión de datos de los archivos csv de Open Contracting Partnership. Data Set del 2018.

Usando el buscador de OpenContracting se buscaron datos sobre Chile (buscador: https://data.open-contracting.org/es/search/). Se decidió descargarlos archivos de los años 2018 y 2021, para tener una muestra de datos que contenga información sobre licitaciones en un rango de tiempo cercano a estas fechas.

In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
%cd Chile
%cd 2018
%ls
%cd ..
%cd ..

c:\Users\camil\OneDrive\Documentos\GitHub\Proyecto-IMT-2200-1-Grupo-14\Datos y trabajo con muestras\Chile
c:\Users\camil\OneDrive\Documentos\GitHub\Proyecto-IMT-2200-1-Grupo-14\Datos y trabajo con muestras\Chile\2018
 El volumen de la unidad C es OS
 El n�mero de serie del volumen es: 240D-CD63

 Directorio de c:\Users\camil\OneDrive\Documentos\GitHub\Proyecto-IMT-2200-1-Grupo-14\Datos y trabajo con muestras\Chile\2018

18-11-2024  15:43    <DIR>          .
18-11-2024  15:05    <DIR>          ..
18-11-2024  15:43        64.571.337 awards.csv
18-11-2024  15:29       137.878.898 awards_documents.csv
18-11-2024  15:29       172.974.592 awards_items.csv
18-11-2024  15:26        30.901.082 awards_suppliers.csv
18-11-2024  15:29       165.819.300 main.csv
18-11-2024  15:31       370.417.597 parties.csv
18-11-2024  15:30       243.504.970 parties_additionalIdentifiers.csv
18-11-2024  15:30       287.507.728 tender_items.csv
18-11-2024  15:27       103.927.462 tender_tenderers.csv
            

## Revision de main.csv

In [3]:
from os import path
import pandas as pd

camino = path.join("Chile","2018","main.csv")
df_main = pd.read_csv(camino, delimiter=",")

In [4]:
df_main.head()

,_link,id,tag,date,ocid,language,initiationType,buyer_id,buyer_name,tender_id,...,tender_tenderPeriod_startDate,tender_tenderPeriod_durationInDays,tender_enquiryPeriod_endDate,tender_enquiryPeriod_startDate,tender_enquiryPeriod_durationInDays,tender_procuringEntity_id,tender_procuringEntity_name,tender_value_amount,tender_value_currency,tender_value_unitOfAccount
0,id-0.0,ocds-70d2nz-4548-142-L117-2018-12-31T11:27:18Z,compiled,2018-12-31T11:27:18Z,ocds-70d2nz-4548-142-L117,es,tender,CL-MP-5464,Ilustre Municipalidad de Quintero | UNIDAD NORMAL,4548-142-L117,...,2017-12-07T16:17:35Z,6.0,2017-12-10T20:04:00Z,2017-12-07T20:04:00Z,3.0,CL-MP-5464,Ilustre Municipalidad de Quintero | UNIDAD NORMAL,NaN,NaN,NaN
1,id-0.1,ocds-70d2nz-2409-1129-L117-2018-12-31T11:28:26Z,compiled,2018-12-31T11:28:26Z,ocds-70d2nz-2409-1129-L117,es,tender,CL-MP-3401,Municipalidad de Los Angeles | MUNICIPALIDAD D...,2409-1129-L117,...,2017-12-18T17:58:24Z,8.0,2017-12-20T14:30:00Z,2017-12-18T19:01:00Z,2.0,CL-MP-3401,Municipalidad de Los Angeles | MUNICIPALIDAD D...,300000.0,CLP,NaN
2,id-0.2,ocds-70d2nz-2961-811-L117-2018-12-31T11:29:05Z,compiled,2018-12-31T11:29:05Z,ocds-70d2nz-2961-811-L117,es,tender,CL-MP-3939,ILUSTRE MUNICIPALIDAD DE LA GRANJA | Adquisici...,2961-811-L117,...,2017-12-19T13:02:43Z,14.0,2017-12-28T17:00:00Z,2017-12-20T09:00:00Z,8.0,CL-MP-3939,ILUSTRE MUNICIPALIDAD DE LA GRANJA | Adquisici...,1452000.0,CLP,NaN
3,id-0.3,ocds-70d2nz-4858-195-L117-2018-12-31T11:21:15Z,compiled,2018-12-31T11:21:15Z,ocds-70d2nz-4858-195-L117,es,tender,CL-MP-5772,Municipalidad de Tirúa | Ilustre Municipalidad...,4858-195-L117,...,2017-12-15T12:23:25Z,6.0,2017-12-18T15:01:00Z,2017-12-15T15:52:00Z,3.0,CL-MP-5772,Municipalidad de Tirúa | Ilustre Municipalidad...,900000.0,CLP,NaN
4,id-0.4,ocds-70d2nz-2409-975-LE17-2018-12-31T11:20:54Z,compiled,2018-12-31T11:20:54Z,ocds-70d2nz-2409-975-LE17,es,tender,CL-MP-3401,Municipalidad de Los Angeles | MUNICIPALIDAD D...,2409-975-LE17,...,2017-12-05T17:11:36Z,6.0,2017-12-06T17:30:00Z,2017-12-05T17:15:00Z,1.0,CL-MP-3401,Municipalidad de Los Angeles | MUNICIPALIDAD D...,6000000.0,CLP,NaN


In [5]:
df_main.columns

Index(['_link', 'id', 'tag', 'date', 'ocid', 'language', 'initiationType',
       'buyer_id', 'buyer_name', 'tender_id',
       'tender_procurementMethodDetails', 'tender_title', 'tender_status',
       'tender_procurementMethod', 'tender_hasEnquiries', 'tender_description',
       'tender_awardPeriod_endDate', 'tender_awardPeriod_startDate',
       'tender_awardPeriod_durationInDays', 'tender_tenderPeriod_endDate',
       'tender_tenderPeriod_startDate', 'tender_tenderPeriod_durationInDays',
       'tender_enquiryPeriod_endDate', 'tender_enquiryPeriod_startDate',
       'tender_enquiryPeriod_durationInDays', 'tender_procuringEntity_id',
       'tender_procuringEntity_name', 'tender_value_amount',
       'tender_value_currency', 'tender_value_unitOfAccount'],
      dtype='object')

In [6]:
df_main["tender_procurementMethodDetails"]

0              Licitación Pública Menor a 100 UTM (L1)
1              Licitación Pública Menor a 100 UTM (L1)
2              Licitación Pública Menor a 100 UTM (L1)
3              Licitación Pública Menor a 100 UTM (L1)
4         Licitación Pública Entre 100 y 1000 UTM (LE)
                              ...                     
244869         Licitación Pública Menor a 100 UTM (L1)
244870         Licitación Pública Menor a 100 UTM (L1)
244871         Licitación Pública Menor a 100 UTM (L1)
244872    Licitación Pública Entre 100 y 1000 UTM (LE)
244873         Licitación Pública Menor a 100 UTM (L1)
Name: tender_procurementMethodDetails, Length: 244874, dtype: object

Se utilizan la pagina <<https://api.mercadopublico.cl/modules/Licitacion.aspx>> para crear el diccionario.

In [7]:

tipos_licitaciones ={'Licitación Pública Menor a 100 UTM (L1)': "L1",
       'Licitación Pública Entre 100 y 1000 UTM (LE)': "LE",
       'Licitación Pública Mayor 1000 UTM (LP)': "LP",
       'Licitación Pública entre a 2000 y 5000 UTM (LQ)':"LQ",
       'Licitación Pública Mayor a 5000 (LR)' :"LR",
       'Licitación Privada entre a 2000 y 5000 UTM (H2)':"H2",
       'Licitación Pública Servicios personales especializados (LS)':"LS",
       "Licitación Privada por Licitación Pública anterior sin oferentes (A1)" : "A1",
       "Licitación Privada por Remanente de Contrato anterior (B1)": "B1",
       "Licitación Privada por Convenios con Personas Jurídicas Extranjeras fuera del Territorio Nacional (E1)": "E1",
       "Licitación Privada por Servicios de Naturaleza Confidencial (F1)": "F1",
       "Licitación Privada por otras causales, excluidas de la ley de Compras (J1)": "J1",
       "Licitación Privada entre 100 y 1000 UTM" :"CO", "Licitación Privada Mayor a 1000 UTM": "B2",
       "Trato Directo por Producto de Licitación Privada anterior sin oferentes o desierta": "A2",
       "Trato Directo por Proveedor Único (D1)": "D1","Licitación Privada Menor a 100 UTM" : "E2", "Trato Directo (Cotización) (C2)" : "C2",
       "Compra Directa (Orden de compra) (C1)" : "C1", "Trato Directo (Cotización) (F2)" :"F2", "Compra Directa (Orden de compra) (F3)" : "F3",
       "Directo (Cotización) (G2)" : "G2", "Compra Directa (Orden de compra) (G1)" : "G1",
       "Orden de Compra menor a 3 UTM (R1)" : "R1", "Orden de Compra sin Resolución (CA)" : "CA", 
       "Orden de Compra proveniente de adquisición sin emisión automática de OC (SE)" : "SE"}


df_main["tender_procurementMethodDetails"] = df_main["tender_procurementMethodDetails"].map(tipos_licitaciones)
df_main["tender_procurementMethodDetails"] = df_main["tender_procurementMethodDetails"].astype("category")

In [8]:
df_main["tender_description"].unique()

array(['DIDECO - NAVIDAD COMUNAL - JUEGOS INFLABLES\nCORRELATIVO GENERAL N° 1721 \nSOLICITUD N° 600 \nDE FECHA 29-11-2017',
       'CONFECCIÓN E INSTALACIÓN DE PROTECCIONES METALICAS EN PUERTAS ESCUELA F-881.\nÍTEM 22.06.001\n\n',
       'sm80 sp3565 servicios generales', ...,
       'Compra de materiales con el cual se hará mantención de alumbrado publico en la comuna.-',
       'Adquisición de Materiales de Construcción',
       'Asesoría Técnica de la Obra para la supervisión de la Ejecución de las obras del\nproyecto denominado Construcción Alcantarillado Sector Pachacamita La Calera.'],
      dtype=object)

In [9]:
df_main["tender_status"].unique()

array(['complete', 'unsuccessful', 'active', 'cancelled', 'planned'],
      dtype=object)

In [10]:
df_main["tender_status"] = df_main["tender_status"].astype("category")

In [11]:
df_main["tender_title"].unique()

array(['DIDECO - NAVIDAD COMUNAL - JUEGOS INFLABLES',
       'CONFECCIÓN E INSTALACIÓN DE PROTECCIONES METÁLICAS EN PUERTAS ESCUELA F-881 ',
       'Reparacion de mamparas', ...,
       'Adquisición de materiales para Canil',
       'MATERIALES PARA MANTENIMIENTO ALUMBRADO PUBLICO.-',
       'ASESORÍA TÉCNICA OBRA ALCANTARILLADO PACHACAMITA'], dtype=object)

In [12]:
df_main.drop(columns=['id', 'tag', 'date', 'ocid', 'language', 'buyer_name', 'tender_id', 'tender_title', 'tender_description', 'tender_procuringEntity_id'], inplace=True)

### Transformación de datos.

In [13]:
df_main["tender_enquiryPeriod_startDate"] = pd.to_datetime(df_main['tender_enquiryPeriod_startDate'], format='%Y-%m-%dT%H:%M:%SZ')
df_main["tender_enquiryPeriod_endDate"] = pd.to_datetime(df_main['tender_enquiryPeriod_endDate'], format='%Y-%m-%dT%H:%M:%SZ')
df_main['tender_tenderPeriod_startDate'] = pd.to_datetime(df_main['tender_tenderPeriod_startDate'], format='%Y-%m-%dT%H:%M:%SZ')

Al parecer hay un error, una de las filas tiene por valor a en esta columna, eliminemosla.

In [14]:
df_main = df_main.drop(df_main[df_main["tender_tenderPeriod_endDate"] == "a"].index)
df_main["tender_tenderPeriod_endDate"] = pd.to_datetime(df_main['tender_tenderPeriod_endDate'], format='%Y-%m-%dT%H:%M:%SZ')
df_main['tender_awardPeriod_startDate'] = pd.to_datetime(df_main['tender_awardPeriod_startDate'], format='%Y-%m-%dT%H:%M:%SZ')

Hay un error en el formato de una fila con contenido <5017-05-05T17:00:00Z>, se eliminará.

In [15]:
df_main.drop(df_main[df_main["tender_awardPeriod_endDate"] == "5017-05-05T17:00:00Z"].index, axis=0, inplace=True)
df_main['tender_awardPeriod_endDate'] = pd.to_datetime(df_main['tender_awardPeriod_endDate'], format='%Y-%m-%dT%H:%M:%SZ')

In [16]:
df_main['tender_enquiryPeriod_durationInDays'] = df_main['tender_enquiryPeriod_endDate'] - df_main['tender_enquiryPeriod_startDate']
df_main['tender_awardPeriod_durationInDays'] = df_main['tender_awardPeriod_endDate'] - df_main['tender_awardPeriod_startDate']
df_main['tender_tenderPeriod_durationInDays'] = df_main['tender_tenderPeriod_endDate'] - df_main['tender_tenderPeriod_startDate']

### Filtro según fecha

Aqui realizaremos un filtro de las licitaciones en base a cuando se iniciaron y cuando terminaron respecto a lo relacionado al 2018. En este caso seleccionaremos las licitaciones cuyo periodo de envio de aplicaciones para determinar la adjudicación comenzó despúes que el 01/09/2017 y la finalización del proceso de adjudicación antes del 01/01/2019.

In [17]:
fecha_filtro = '2017-09-01'
df_filtrado = df_main.loc[df_main['tender_tenderPeriod_startDate'] >= fecha_filtro]
fecha_filtro = '2019-01-01'
df_filtrado = df_filtrado.loc[df_filtrado['tender_awardPeriod_endDate'] < fecha_filtro]

In [18]:
df_filtrado.reset_index(drop=True, inplace=True)
df_filtrado.head()

,_link,initiationType,buyer_id,tender_procurementMethodDetails,tender_status,tender_procurementMethod,tender_hasEnquiries,tender_awardPeriod_endDate,tender_awardPeriod_startDate,tender_awardPeriod_durationInDays,tender_tenderPeriod_endDate,tender_tenderPeriod_startDate,tender_tenderPeriod_durationInDays,tender_enquiryPeriod_endDate,tender_enquiryPeriod_startDate,tender_enquiryPeriod_durationInDays,tender_procuringEntity_name,tender_value_amount,tender_value_currency,tender_value_unitOfAccount
0,id-0.0,tender,CL-MP-5464,L1,complete,open,False,2017-12-15 18:05:00,2017-12-13 09:03:00,2 days 09:02:00,2017-12-13 09:02:00,2017-12-07 16:17:35,5 days 16:44:25,2017-12-10 20:04:00,2017-12-07 20:04:00,3 days 00:00:00,Ilustre Municipalidad de Quintero | UNIDAD NORMAL,NaN,NaN,NaN
1,id-0.1,tender,CL-MP-3401,L1,unsuccessful,open,False,2018-01-25 11:35:00,2017-12-26 15:31:00,29 days 20:04:00,2017-12-26 15:30:00,2017-12-18 17:58:24,7 days 21:31:36,2017-12-20 14:30:00,2017-12-18 19:01:00,1 days 19:29:00,Municipalidad de Los Angeles | MUNICIPALIDAD D...,300000.0,CLP,NaN
2,id-0.3,tender,CL-MP-5772,L1,unsuccessful,open,False,2017-12-26 15:53:00,2017-12-21 10:03:00,5 days 05:50:00,2017-12-21 10:02:00,2017-12-15 12:23:25,5 days 21:38:35,2017-12-18 15:01:00,2017-12-15 15:52:00,2 days 23:09:00,Municipalidad de Tirúa | Ilustre Municipalidad...,900000.0,CLP,NaN
3,id-0.4,tender,CL-MP-3401,LE,complete,open,True,2017-12-14 16:55:00,2017-12-11 16:30:00,3 days 00:25:00,2017-12-11 16:00:00,2017-12-05 17:11:36,5 days 22:48:24,2017-12-06 17:30:00,2017-12-05 17:15:00,1 days 00:15:00,Municipalidad de Los Angeles | MUNICIPALIDAD D...,6000000.0,CLP,NaN
4,id-0.5,tender,CL-MP-5748,L1,complete,open,True,2017-12-29 16:00:00,2017-12-28 12:02:00,1 days 03:58:00,2017-12-28 12:00:00,2017-12-21 16:27:16,6 days 19:32:44,2017-12-26 12:00:00,2017-12-22 19:00:00,3 days 17:00:00,Ilustre Municipalidad de Cochamo | Ilustre Mun...,NaN,NaN,NaN


In [19]:
df_filtrado.rename(columns={"tender_value_amount":"max_estimated_cost", "tender_value_currency": "max_estimated_cost_currency", "tender_value_unitOfAccount": "max_estimated_cost_unitOfAccount"}, inplace=True )

In [20]:
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31067 entries, 0 to 31066
Data columns (total 20 columns):
 #   Column                               Non-Null Count  Dtype          
---  ------                               --------------  -----          
 0   _link                                31067 non-null  object         
 1   initiationType                       31067 non-null  object         
 2   buyer_id                             31067 non-null  object         
 3   tender_procurementMethodDetails      30762 non-null  category       
 4   tender_status                        31067 non-null  category       
 5   tender_procurementMethod             31067 non-null  object         
 6   tender_hasEnquiries                  31067 non-null  bool           
 7   tender_awardPeriod_endDate           31067 non-null  datetime64[ns] 
 8   tender_awardPeriod_startDate         31067 non-null  datetime64[ns] 
 9   tender_awardPeriod_durationInDays    31067 non-null  timedelta64[ns]
 10

In [21]:
df_filtrado.loc[df_filtrado["tender_value_amount"].isna()].info()

KeyError: 'tender_value_amount'

In [ ]:
def crear_unit_value_monetary_unit(indice:int, comparador=np.nan) -> str:
    if df_filtrado.iloc[indice]["tender_value_currency"] == comparador:
        if (df_filtrado.iloc[indice]["tender_status"] == 'unsuccessful') or (df_filtrado.iloc[indice]["tender_status"] ==  'cancelled') or (df_filtrado.iloc[indice]["tender_status"] == 'pending'):
            return ("No aplica")
        return df_filtrado.iloc[indice]["tender_value_unitOfAccount"]
    return df_filtrado.iloc[indice]["tender_value_currency"]

## Revision awards.csv

In [ ]:
camino = path.join("Chile","2018","awards.csv")
df_awards = pd.read_csv(camino, delimiter=",")

In [ ]:
df_awards.head()

,_link,_link_main,id,date,description,title,status,value_amount,value_currency,value_unitOfAccount
0,id-0.0.awards.0,id-0.0,8270925,2017-12-23T13:23:24Z,DIDECO - NAVIDAD COMUNAL - JUEGOS INFLABLES\r\...,DIDECO - NAVIDAD COMUNAL - JUEGOS INFLABLES,active,690000.0,CLP,NaN
1,id-0.1.awards.0,id-0.1,8274205,NaN,CONFECCIÓN E INSTALACIÓN DE PROTECCIONES METAL...,CONFECCIÓN E INSTALACIÓN DE PROTECCIONES METÁL...,unsuccessful,NaN,NaN,NaN
2,id-0.2.awards.0,id-0.2,8275841,2018-02-02T11:16:54Z,sm80 sp3565 servicios generales,Reparacion de mamparas,active,870000.0,CLP,NaN
3,id-0.3.awards.0,id-0.3,8254526,2017-12-26T18:19:33Z,VISITA A HUERTO ORGANICO DE BERRIES HUBICADO E...,VISTA A HUERTO ORGANICO DE BERRIES HUBICADO EN...,unsuccessful,NaN,NaN,NaN
4,id-0.4.awards.0,id-0.4,8253535,2017-12-21T18:17:23Z,CAPACITACIÓN PARA DOCENTES DE LA ESC. F-929 - ...,CAPACITACIÓN PARA DOCENTES DE LA ESC. F-929 - ...,active,6000000.0,CLP,NaN


In [ ]:
for columna in df_awards.columns:
    print(columna)
    print("#####################################################################")
    print(df_awards[columna].unique())

_link
#####################################################################
['id-0.0.awards.0' 'id-0.1.awards.0' 'id-0.2.awards.0' ...
 'id-15.15078.awards.0' 'id-15.15079.awards.0' 'id-15.15080.awards.0']
_link_main
#####################################################################
['id-0.0' 'id-0.1' 'id-0.2' ... 'id-15.15078' 'id-15.15079' 'id-15.15080']
id
#####################################################################
[8270925 8274205 8275841 ... 7874241 7880005 7879330]
date
#####################################################################
['2017-12-23T13:23:24Z' nan '2018-02-02T11:16:54Z' ...
 '2016-03-21T15:44:13Z' '2016-05-30T19:57:32Z' '2016-02-19T16:06:59Z']
description
#####################################################################
['DIDECO - NAVIDAD COMUNAL - JUEGOS INFLABLES\r\nCORRELATIVO GENERAL N° 1721 \r\nSOLICITUD N° 600 \r\nDE FECHA 29-11-2017'
 'CONFECCIÓN E INSTALACIÓN DE PROTECCIONES METALICAS EN PUERTAS ESCUELA F-881.\r\nÍTEM 22.06.001\r\n\r\n'

In [ ]:
df_no_nan_un_ac = df_awards.dropna(subset=["value_unitOfAccount"])[["value_currency", "value_unitOfAccount", "status"]]
df_unit_of_account = df_awards.loc[df_awards["value_currency"].isna()][["value_currency", "value_unitOfAccount", "status"]]

In [ ]:
df_unit_of_account["value_unitOfAccount"].describe()

count      83
unique      1
top       UTM
freq       83
Name: value_unitOfAccount, dtype: object

In [ ]:
df_no_nan_un_ac["value_unitOfAccount"].describe()

count      83
unique      1
top       UTM
freq       83
Name: value_unitOfAccount, dtype: object

In [ ]:
print(len(df_unit_of_account), len(df_no_nan_un_ac))

62653 83


In [ ]:
df_awards["value_monetary_unit"] = [crear_unit_value_monetary_unit(indice) for indice in df_awards.index]

In [ ]:
df_awards["value_monetary_unit"].unique()

array(['CLP', nan, 'CLF', 'USD', 'EUR'], dtype=object)

In [ ]:
df_awards.loc[df_awards["status"] == "unsuccessful"]

,_link,_link_main,id,date,description,title,status,value_amount,value_currency,value_unitOfAccount,value_monetary_unit
1,id-0.1.awards.0,id-0.1,8274205,NaN,CONFECCIÓN E INSTALACIÓN DE PROTECCIONES METAL...,CONFECCIÓN E INSTALACIÓN DE PROTECCIONES METÁL...,unsuccessful,NaN,NaN,NaN,NaN
3,id-0.3.awards.0,id-0.3,8254526,2017-12-26T18:19:33Z,VISITA A HUERTO ORGANICO DE BERRIES HUBICADO E...,VISTA A HUERTO ORGANICO DE BERRIES HUBICADO EN...,unsuccessful,NaN,NaN,NaN,NaN
8,id-0.8.awards.0,id-0.8,8257363,NaN,CONTRATO DE BUS PARA GIRA DE ESTUDIOS DE ESCUE...,CONTRATO BUS GIRA DE ESTUDIOS ESCUELA G-150,unsuccessful,NaN,NaN,NaN,NaN
9,id-0.9.awards.0,id-0.9,8259052,2018-01-30T11:15:26Z,MEGAFONOS 25 UNID. PREVENCIONISTA M:114,MEGAFONOS 25 UNID. PREVENCIONISTA M:114,unsuccessful,NaN,NaN,NaN,NaN
16,id-0.16.awards.0,id-0.16,8274557,2018-02-02T14:55:54Z,Compra de Servicios de Transporte Para Pacient...,TRANSPORTE PARA PACIENTES Y FUNCIONARIOS,unsuccessful,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
239975,id-15.15035.awards.0,id-15.15035,7874833,2016-03-28T13:12:59Z,"“EL HOSPITAL”, llama a licitación pública para...",ADQUISICIÓN DE INSUMOS MÉDICOS PARA TERAPÍA OC...,unsuccessful,NaN,NaN,NaN,NaN
239984,id-15.15044.awards.0,id-15.15044,7843245,NaN,contar con los insumos necesarios para la pron...,CONVENIO ANUAL SISTEMA DE EXTRACCION DE ARTERI...,unsuccessful,NaN,NaN,NaN,NaN
239991,id-15.15051.awards.0,id-15.15051,7884964,NaN,LA NECESIDAD DE LICITAR LA CONTRATACIÓN DE SER...,SERVICIO DE TRASLADO ALUMNOS - ESCUELA ITALIA.,unsuccessful,NaN,NaN,NaN,NaN
239998,id-15.15058.awards.0,id-15.15058,7874802,NaN,"Segun correo de la Jefa enfermera SAMU, solici...",Solicita entrega de almuerzos y cenas,unsuccessful,NaN,NaN,NaN,NaN


In [ ]:
df_awards.drop(columns=['id', 'description', 'title', "value_unitOfAccount", "value_currency", "date"],inplace=True)

## Revisión de awards_items.csv

In [ ]:
camino = path.join("Chile","2018","awards_items.csv") 
df_awards_items = pd.read_csv(camino, delimiter=",")

In [ ]:
df_awards_items.columns

In [ ]:
df_awards_items.info()

In [ ]:
df_awards_items["unit_value_unitOfAccount"].describe()

In [ ]:
df_awards_items.loc[df_awards_items["unit_value_unitOfAccount"] == "UTM"][["unit_value_unitOfAccount","unit_value_currency"]]

In [ ]:
df_unit_of_account = df_awards_items.loc[df_awards_items["unit_value_currency"].isna()][["unit_value_currency", "unit_value_unitOfAccount"]]
df_no_nan_un_ac = df_awards_items.dropna(subset=["unit_value_unitOfAccount"])[["unit_value_currency", "unit_value_unitOfAccount"]]

In [ ]:
print(df_no_nan_un_ac.index == df_unit_of_account.index)

Podemos observar que para los valores de <unit_value_currency> que son NaN existe un dato en <unit_value_unitOfAccount>, por lo que crearemos una nueva columna llamada <unit_value_monetary_unit> que tenga los valores de ambas columna según correspondan.

In [ ]:
def crear_unit_value_monetary_unit(indice:int, comparador=np.nan) -> str:
    if df_awards_items.iloc[indice]["unit_value_currency"] == comparador:
        return df_awards_items.iloc[indice]["unit_value_unitOfAccount"]
    return df_awards_items.iloc[indice]["unit_value_currency"]

In [ ]:
df_awards_items["unit_value_monetary_unit"] = [crear_unit_value_monetary_unit(indice) for indice in df_awards_items.index]

In [ ]:
df_awards_items["unit_value_monetary_unit"].describe()

In [ ]:
df_awards_items.drop(columns=['classification_uri', 'classification_scheme','unit_value_unitOfAccount', "unit_value_currency" ], inplace=True)

In [ ]:
df_award_and_items = df_awards.merge(df_awards_items, left_on="_link", right_on="_link_awards")
df_awards_items.head()

## Revisión de awards_suppliers.csv

In [ ]:
camino = path.join("Chile","2018","awards_suppliers.csv")
df_awards_supplier = pd.read_csv(camino, delimiter=",")

In [ ]:
df_awards_supplier.info()

In [ ]:
df_awards_supplier

In [ ]:
df_award_and_supplier = df_awards.merge(df_awards_supplier, left_on="_link", right_on="_link_awards")
df_award_and_supplier.head()

## Revisión de awards_documents.csv

In [ ]:
camino = path.join("Chile","2018", "awards_documents.csv")
df_award_documents = pd.read_csv(camino, sep=",")
df_award_documents.head()

In [ ]:
df_award_documents.info()

No se utilizará este DataFrame, puesto que no contiene información relevante para lo que se busca resolver.

## Revisión de parties.csv

In [ ]:
camino = path.join("Chile","2018", "parties.csv")
df_parties = pd.read_csv(camino, sep=",")
df_parties.head()

In [ ]:
df_parties.info()

In [ ]:
df_parties.columns

In [ ]:
df_parties.drop(columns=['address_streetAddress', "identifier_id", 'identifier_scheme','identifier_legalName', 'contactPoint_name', 'contactPoint_email','contactPoint_telephone', 'contactPoint_faxNumber'], inplace=True)

Se eliminan estas columnas porque ofrecen información que no es de interes, ni se puede utilizar para obtener información en alguna de las paginas relacionadas al dataset.

In [ ]:
df_parties.info()

## Revisión de parties_additionalIdentifiers.csv

In [ ]:
camino = path.join("Chile","2018", "parties_additionalIdentifiers.csv")
df_parties_add = pd.read_csv(camino, sep=",")
df_parties_add.head()

In [ ]:
df_parties_add.iloc[3]["uri"]

No se realizará merge, puesto que no tiene información adicional que sea relevante y no se puede utilizar

### Revisión de tender_items.csv

In [ ]:
camino = path.join("Chile","2018", "tender_items.csv")
df_tender_item = pd.read_csv(camino, sep=",")
df_tender_item.head()

In [ ]:
df_tender_item.info()

No se utilizara este dataframe en un merge, porque ya se tiene un dataframe más completo para obtenerla información que no se tiene en el df_filtrado   

### Revisión de tender_tenderers.csv

In [ ]:
camino = path.join("Chile","2018", "tender_tenderers.csv")
df_tender_tenderers = pd.read_csv(camino, sep=",")

In [ ]:
df_tender_tenderers.head()

In [ ]:
df_tender_tenderers.info()

No se utilizará este dataframe, puesto que ya se tiene toda la información que entrega

### Clasificación con UNSPCS.

In [ ]:
df_clasificaciones = pd.read_excel("unspcs-clasificador-de-bienes-y-servicios-de-naciones-unidas-en-espanol.xlsx", skiprows=5)
df_clasificaciones.head(10)

El dataframe df_clasificaciones corresponde a un conjunto de datos que posee el significado de cada objeto solicitados según la codificación UNSPSC, por lo que se hara merge con el df_main para saber que se esta comprando en cada licitación, se desecharán algunas de las columnas de df_clasificaciones para esto.

In [ ]:
df_clasificaciones.drop(columns=["Código segmento","Código Familia","Nombre Familia","Nombre Clase","Código Clase"], inplace=True)

In [ ]:
df_2018 = df_main

In [ ]:
df_2018.columns

Se revisarán que las filas no posean solo un dato, si sucede, se eliminarán. (Se ignorarán las filas que evidentemente poseen más de un dato).

In [ ]:
df_iguales = pd.DataFrame(df_2018["tender_value_amount"] == df_2018["value_amount"])
df_iguales.head()

In [ ]:
df_iguales.rename(columns={0: "tender_value_amount == value_amount"}, inplace=True)

In [ ]:
df_iguales["tender_value_amount == unit_value_amount"] = df_2018["tender_value_amount"] == df_2018["unit_value_amount"]
df_iguales["value_amount == unit_value_amount"] = df_2018["value_amount"] == df_2018["unit_value_amount"]

df_iguales["tender_value_currency == value_currency"] = df_2018["tender_value_currency"] == df_2018["value_currency"]
df_iguales["tender_value_currency == unit_value_currency"] = df_2018["tender_value_currency"] == df_2018["unit_value_currency"]
df_iguales["unit_value_currency == value_currency"] = df_2018["unit_value_currency"] == df_2018["value_currency"]
df_iguales["buyer == procuring_entity"] = df_2018["buyer_name"] == df_2018["tender_procuringEntity_name"]

In [ ]:
for columna in df_iguales.columns:
    print(columna, df_iguales[columna].unique())

In [ ]:
df_2018.drop(columns=["buyer_name"], inplace=True)

Se observa que solo las columnas buyer y procuring_entity son iguales entre si, por lo que se eliminará buyer_name.

In [ ]:
duplicates = df_2018.duplicated(keep=False)
dups = df_2018[duplicates]
dups.head(10)

Se tiene en consideración que dentro de un mismo proceso de licitación pueden existir diferentes entidades que ofrecen cubrir las necesidades planteadas al publicar la licitación, por lo que se mantendran los duplicados.

Se cambiarán algunas columnas a category.

In [ ]:
for columna in df_2018.columns:
    print("\n")
    print(columna)
    print("\n")
    print(df_2018[columna].unique())
    print("\n")
    print("########################################################################################")

## Limpieza de datos nulos

Se revisan filas con valores nulos.

In [ ]:
Datos_with_NaN = df_filtrado.loc[:, df_filtrado.isna().any()]

In [ ]:
print(f"La cantidad de datos NaN es de: {len(Datos_with_NaN[Datos_with_NaN.isna().any(axis=1)])} en todo el dataframe de {len(df_filtrado)}")

In [ ]:
df_filtrado[['tender_value_amount','tender_value_currency', 'value_amount', 'value_currency', 'quantity','unit_name', 'unit_value_amount', 'unit_value_currency']].info()


In [ ]:
nan_tender_v = df_filtrado[df_filtrado['value_currency'].isna()]
nan_tender_v[["value_currency", "value_amount","unit_value_currency", "unit_value_amount","tender_value_amount", "tender_value_currency", "status", "tender_hasEnquiries"]]

In [ ]:
nan_tender_v["status"].unique()

In [ ]:
non_nan_t= df_filtrado.loc[df_filtrado.dropna(subset="value_currency").index]
non_nan_t[["value_currency", "value_amount","unit_value_currency", "unit_value_amount","tender_value_amount", "tender_value_currency", "status", "tender_hasEnquiries"]].head()

In [ ]:
non_nan_t["status"].unique()

Como se puede observar los casos con valores NaN en valores pertinentes al costo de la licitación se ven relacionados a que el status de la licitación <unsuccessful> por lo que se cambiara el valor NaN para estos casos por "Licitación no completada", para no tener problemas en eliminar otras filas.

In [ ]:
df_filtrado["value_currency"] = df_filtrado["value_currency"].replace(np.nan, "No aplica")
df_filtrado["unit_value_currency"] = df_filtrado["unit_value_currency"].replace(np.nan, "No aplica")
df_filtrado["unit_value_amount"] = df_filtrado["unit_value_amount"].replace(np.nan, "No aplica")
df_filtrado["value_amount"] = df_filtrado["value_amount"].replace(np.nan, "No aplica")
df_filtrado["tender_value_currency"] = df_filtrado["tender_value_currency"].replace(np.nan, "No aplica")
df_filtrado["tender_value_amount"] = df_filtrado["tender_value_amount"].replace(np.nan, "No aplica")
df_filtrado["Award_supplier_name"] = df_filtrado["Award_supplier_name"]

In [ ]:
df_filtrado["value_currency"] = df_filtrado["value_currency"].astype("category")
df_filtrado["unit_value_currency"] = df_filtrado["unit_value_currency"].astype("category")
df_filtrado["tender_value_currency"] = df_filtrado["tender_value_currency"].astype("category")

En base a esta información se vera según casos los valores NaN.

In [ ]:
df_filtrado[["tender_tenderPeriod_durationInDays", "tender_awardPeriod_durationInDays", "tender_enquiryPeriod_durationInDays"]].info()

In [ ]:
df_filtrado.info()

Teniendo en cuenta que ya resolvimos los problemas que pueden causar las licitaciones no completadas, podemos realizar un dropna.

In [ ]:
df_filt_2018 = df_filtrado.dropna(subset=["status"])
df_filt_2018.info()

In [ ]:
df_filtrado.loc[df_filtrado["link"].isna().index].info()